In [ ]:
pip install pm4py

In [ ]:
import warnings
warnings.filterwarnings('ignore')

###Importing Data from drive

In [ ]:
import os
CURR_DIR = os.getcwd()
for subdir, dirs, files in os.walk('/content/drive/My Drive/Connector'):
    print(dirs)
isExist = os.path.exists('/content/drive/My Drive/Connector')
print(isExist)

First round data

In [ ]:
import os
Model_directory1 = '/content/drive/My Drive/Connector/Models/'

file_list_model1 = []   # define file_list to save all csv files
for subdir, dirs, files in os.walk(Model_directory1):
    for file in files:
        if file.endswith('.csv'):
            file_list_model1.append(Model_directory1+file)   # save the filenames in file_list

file_list_model1


In [ ]:
log_directory1 = '/content/drive/My Drive/Connector/Logs/'

file_list_log1 = []   # define file_list to save all csv files
for subdir, dirs, files in os.walk(log_directory1):
    for file in files:
        if file.endswith('.csv'):
            file_list_log1.append(log_directory1+file)   # save the filenames in file_list

file_list_log1

Second round data

In [ ]:
import os
Model_directory = '/content/drive/My Drive/Top10Connectors/train/'
print(list(os.walk(Model_directory)))
file_list_model = []   # define file_list to save all csv files
for subdir, dirs, files in os.walk(Model_directory):
    
    for file in files:
        if file.endswith('.csv'):
            file_list_model.append(Model_directory+file)   # save the filenames in file_list

file_list_model
print(len(file_list_model))

In [ ]:
log_directory = '/content/drive/My Drive/Top10Connectors/test/'

file_list_log = []   # define file_list to save all csv files
for subdir, dirs, files in os.walk(log_directory):
    for file in files:
        if file.endswith('.csv'):
            file_list_log.append(log_directory+file)   # save the filenames in file_list

file_list_log

###Add libraries for PM4PY

In [ ]:
import pandas as pd
import pm4py
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.inductive.variants import im_f
from pm4py.algo.discovery.inductive import parameters
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
from pm4py.algo.conformance.tokenreplay import algorithm as token_replay

##Create functions for process discovery and conformance checking

####Function for Inductive miner and alignment conformance checking algorithms

In [ ]:
def Conf_ind_align(model,log):
    model = pd.read_csv(model, sep=',',encoding='latin1')
    log = pd.read_csv(log, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]
    log_csv= log[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)

    log_csv.columns = cols
    log_csv['time:timestamp'] = pd.to_datetime(log_csv['time:timestamp'])
    log_csv['concept:name'] = log_csv['concept:name'].astype(str)

    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)
    log = log_converter.apply(log_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    # process discovery using Inductive Miner
    net, initial_marking, final_marking = inductive_miner.apply(model)
    # Conformance checking using alignments
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking)

    df = pd.DataFrame(aligned_traces)
    dff = df["fitness"].mean()

    return df

####Function for Inductive infrequent miner with default parameters and alignment conformance checking algorithms



In [ ]:
def Conf_imf_align(model,log):
    model = pd.read_csv(model, sep=',',encoding='latin1')
    log = pd.read_csv(log, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]
    log_csv= log[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)

    log_csv.columns = cols
    log_csv['time:timestamp'] = pd.to_datetime(log_csv['time:timestamp'])
    log_csv['concept:name'] = log_csv['concept:name'].astype(str)

    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)
    log = log_converter.apply(log_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    net, initial_marking, final_marking = im_f.algorithm.apply(model,parameters=None)
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking)

    df = pd.DataFrame(aligned_traces)
    dff = df["fitness"].mean()
    return dff

####Function for Inductive infrequent miner with "noiseThreshold" : 0.5 and alignment conformance checking algorithms 

In [ ]:
def Conf_imf_align_NT(model,log):
    model = pd.read_csv(model, sep=',',encoding='latin1')
    log = pd.read_csv(log, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]
    log_csv= log[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)

    log_csv.columns = cols
    log_csv['time:timestamp'] = pd.to_datetime(log_csv['time:timestamp'])
    log_csv['concept:name'] = log_csv['concept:name'].astype(str)

    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)
    log = log_converter.apply(log_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    net, initial_marking, final_marking = im_f.algorithm.apply(model,parameters={"noiseThreshold" : 0.5})
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking)

    df = pd.DataFrame(aligned_traces)
    dff = df["fitness"].mean()
    return df

####Function for Heuristic miner with default parameters and alignment conformance checking algorithms


In [ ]:
def Conf_Heu_alig(model,log):
    model = pd.read_csv(model, sep=',',encoding='latin1')
    log = pd.read_csv(log, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]
    log_csv= log[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)

    log_csv.columns = cols
    log_csv['time:timestamp'] = pd.to_datetime(log_csv['time:timestamp'])
    log_csv['concept:name'] = log_csv['concept:name'].astype(str)

    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)
    log = log_converter.apply(log_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    net, initial_marking, final_marking = heuristics_miner.apply(model,parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.99})
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking)

    df = pd.DataFrame(aligned_traces)
    dff = df["fitness"].mean()
    return dff




####Function for Inductive miner and token-based replay conformance checking algorithms

In [ ]:
def Conf_ind_token(model,log):
    model = pd.read_csv(model, sep=',',encoding='latin1')
    log = pd.read_csv(log, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]
    log_csv= log[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)

    log_csv.columns = cols
    log_csv['time:timestamp'] = pd.to_datetime(log_csv['time:timestamp'])
    log_csv['concept:name'] = log_csv['concept:name'].astype(str)

    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)
    log = log_converter.apply(log_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    # process discovery using Inductive Miner
    net, initial_marking, final_marking = inductive_miner.apply(model)
    # Conformance checking using token-based replay
    aligned_traces = token_replay.apply(log, net, initial_marking, final_marking)

    df = pd.DataFrame(aligned_traces)
    dff = df["trace_fitness"].mean()

    return dff

####Function for Inductive infrequent miner with default parameters and token-based replay conformance checking algorithms

In [ ]:
def Conf_imf_token(model,log):
    model = pd.read_csv(model, sep=',',encoding='latin1')
    log = pd.read_csv(log, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]
    log_csv= log[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)

    log_csv.columns = cols
    log_csv['time:timestamp'] = pd.to_datetime(log_csv['time:timestamp'])
    log_csv['concept:name'] = log_csv['concept:name'].astype(str)

    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)
    log = log_converter.apply(log_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    # process discovery using Inductive Miner Infrequent
    net, initial_marking, final_marking = im_f.algorithm.apply(model,parameters=None)
    # Conformance checking using token-based replay
    aligned_traces = token_replay.apply(log, net, initial_marking, final_marking)

    df = pd.DataFrame(aligned_traces)
    dff = df["trace_fitness"].mean()
    return dff

####Function for Inductive infrequent miner with "noiseThreshold" : 0.5 and token-based replay conformance checking algorithms 

In [ ]:
def Conf_imf_token_NT(model,log):
    model = pd.read_csv(model, sep=',',encoding='latin1')
    log = pd.read_csv(log, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]
    log_csv= log[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)

    log_csv.columns = cols
    log_csv['time:timestamp'] = pd.to_datetime(log_csv['time:timestamp'])
    log_csv['concept:name'] = log_csv['concept:name'].astype(str)

    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)
    log = log_converter.apply(log_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    net, initial_marking, final_marking = im_f.algorithm.apply(model,parameters={"noiseThreshold" : 0.5})
    # Conformance checking using token-based replay
    aligned_traces = token_replay.apply(log, net, initial_marking, final_marking)

    df = pd.DataFrame(aligned_traces)
    dff = df["trace_fitness"].mean()
    return dff

####Function for Heuristic miner with default parameters and token-based replay conformance checking algorithms

In [ ]:
def Conf_Heu_token(model,log):
    model = pd.read_csv(model, sep=',',encoding='latin1')
    log = pd.read_csv(log, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]
    log_csv= log[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)

    log_csv.columns = cols
    log_csv['time:timestamp'] = pd.to_datetime(log_csv['time:timestamp'])
    log_csv['concept:name'] = log_csv['concept:name'].astype(str)

    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)
    log = log_converter.apply(log_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    net, initial_marking, final_marking = heuristics_miner.apply(model,parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.99})
    # Conformance checking using token-based replay
    aligned_traces = token_replay.apply(log, net, initial_marking, final_marking)

    df = pd.DataFrame(aligned_traces)
    dff = df["trace_fitness"].mean()
    return dff




## Train and get the Results

### First test

In [ ]:
y = file_list_model1
x = file_list_log1
data_first=[]
for i in file_list_model1:
  row=[]
  for j in file_list_log1:
     row.append(Conf_ind_align(i,j))
  data_first.append(row)
  

df_first1 = pd.DataFrame(data_first, index=y, columns=x)
df_first1

In [ ]:
y = file_list_model1
x = file_list_log1
data_first=[]
for i in file_list_model1:
  row=[]
  for j in file_list_log1:
     row.append(Conf_imf_align(i,j))
  data_first.append(row)
  

df_first1 = pd.DataFrame(data_first, index=y, columns=x)
df_first1
df_max = df_first1.max()
df_max = pd.DataFrame(df_max)
df_max.columns=['MAX']
df_max=df_max.transpose()
df = df_first1.append(df_max)
df

In [ ]:
y = file_list_model
x = file_list_log
data_first0=[]
for i in file_list_model:
  row=[]
  for j in file_list_log:
     row.append(Conf_imf_align_NT(i,j))
  data_first0.append(row)
  

df_first0 = pd.DataFrame(data_first0, index=y, columns=x)
df_first0

In [ ]:
y = file_list_model1
x = file_list_log1
data_first=[]
for i in file_list_model1:
  row=[]
  for j in file_list_log1:
     row.append("%.4f" %Conf_imf_token(i,j))
  data_first.append(row)
  

df_first1 = pd.DataFrame(data_first, index=y, columns=x)
df_first1

In [ ]:
y = file_list_model1
x = file_list_log1
data_first=[]
for i in file_list_model1:
  row=[]
  for j in file_list_log1:
     row.append("%.4f" %Conf_ind_token(i,j))
  data_first.append(row)
  

df_first1 = pd.DataFrame(data_first, index=y, columns=x)
df_first1
df_max = df_first1.max()
df_max = pd.DataFrame(df_max)
df_max.columns=['MAX']
df_max=df_max.transpose()
df = df_first1.append(df_max)
df

In [ ]:
y = file_list_model1
x = file_list_log1
data_first=[]
for i in file_list_model1:
  row=[]
  for j in file_list_log1:
     row.append("%.4f" %Conf_Heu_alig(i,j))
  data_first.append(row)
  

df_first1 = pd.DataFrame(data_first, index=y, columns=x)
df_first1
df_max = df_first1.max()
df_max = pd.DataFrame(df_max)
df_max.columns=['MAX']
df_max=df_max.transpose()
df = df_first1.append(df_max)
df

In [ ]:
y = file_list_model1
x = file_list_log1
data_first=[]
for i in file_list_model1:
  row=[]
  for j in file_list_log1:
     row.append("%.4f" %Conf_Heu_token(i,j))
  data_first.append(row)
  

df_first1 = pd.DataFrame(data_first, index=y, columns=x)
df_first1
df_max = df_first1.max()
df_max = pd.DataFrame(df_max)
df_max.columns=['MAX']
df_max=df_max.transpose()
df = df_first1.append(df_max)
df

In [ ]:
y = file_list_model
x = file_list_log
data_first=[]
for i in file_list_model:
  row=[]
  for j in file_list_log:
     row.append("%.4f" %Conf_Heu_alig(i,j))
  data_first.append(row)
  

df_first1 = pd.DataFrame(data_first, index=y, columns=x)
df_first1
df_max = df_first1.max()
df_max = pd.DataFrame(df_max)
df_max.columns=['MAX']
df_max=df_max.transpose()
df = df_first1.append(df_max)
df

In [ ]:
def imf_NT(model):
    model = pd.read_csv(model, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)


    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    net, initial_marking, final_marking = im_f.algorithm.apply(model,parameters={"noiseThreshold" : 0.5})

    return net, initial_marking, final_marking

In [ ]:
y = file_list_model
models=[]
for i in file_list_model:
     models.append(imf_NT(i))



In [ ]:
print(models)

[(places: [ p_10, p_12, p_13, p_14, p_15, p_16, p_19, p_20, p_23, p_24, p_25, p_27, p_28, p_29, p_3, p_31, p_32, p_4, p_5, p_6, p_7, p_8, p_9, sink, source ]
transitions: [ (024318eb-a350-411d-a638-f069b05f1069, 'firefoxPopulate text field in window'), (10da3ee0-37f9-4d86-b085-3e5161648ad7, 'ShellExperienceHostClick UI element in window'), (1ae85f41-0c6b-42c6-8d27-ac1eaed22fa0, 'msedgePopulate text field in window'), (2ee79cd6-6bbd-4bb5-854a-e97a95532750, 'chromeSelect tab in window'), (3e7f3155-4498-4b7d-bb70-5d5a48815201, 'chromeSelect menu option in window'), (51438e1a-881b-463f-b7b2-bef79846f275, 'msedgeClick UI element in window'), (582e75e0-5e09-45c2-a328-485dfebb7b07, 'Press button in window'), (5c11d18e-161a-42af-af68-8a200fad23f1, 'firefoxPress button in window'), (6f306cf1-b998-4dcf-8536-50dcb7f9c380, 'firefoxClick UI element in window'), (6f690303-4ad1-431a-a4e7-2a3b5468bc85, 'chromeClick UI element in window'), (73a14945-d547-401c-a5e7-992082616aa8, 'chromePress button in w

In [ ]:
print(models[0][0].places)
print(models[0][0].transitions)
print(models[0][0].arcs)

{p_20, p_9, p_14, source, p_6, p_8, p_10, p_23, p_24, p_25, p_31, p_19, p_29, p_27, p_28, p_13, p_15, p_12, p_16, p_4, sink, p_5, p_3, p_7, p_32}
{(skip_28, None), (skip_24, None), (init_loop_26, None), (10da3ee0-37f9-4d86-b085-3e5161648ad7, 'ShellExperienceHostClick UI element in window'), (84cc4691-c298-49dc-b345-1c56e0ddd7f1, 'OUTLOOKPress button in window'), (init_loop_5, None), (skip_4, None), (1ae85f41-0c6b-42c6-8d27-ac1eaed22fa0, 'msedgePopulate text field in window'), (tau_1, None), (5c11d18e-161a-42af-af68-8a200fad23f1, 'firefoxPress button in window'), (skip_2, None), (e23256de-a276-425d-904a-133ba031005e, 'Click UI element in window'), (a9d235c0-1cd7-4fb2-a1a2-0feb3e71ff63, 'chromePopulate text field in window'), (8c0b84f3-bcf3-4319-8eb9-192aa3593938, 'msedgePress button in window'), (73a14945-d547-401c-a5e7-992082616aa8, 'chromePress button in window'), (3e7f3155-4498-4b7d-bb70-5d5a48815201, 'chromeSelect menu option in window'), (skip_25, None), (skip_34, None), (skip_3, N

In [ ]:
with open(r'models.txt', 'w') as fp:
    for item in models:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
def Conf_imf_align_NT(model,log):

    model = pd.read_csv(model, sep=',',encoding='latin1')
    model_csv= model[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']
    model_csv.columns = cols
    model_csv['time:timestamp'] = pd.to_datetime(model_csv['time:timestamp'])
    model_csv['concept:name'] = model_csv['concept:name'].astype(str)


    model = log_converter.apply(model_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    net, initial_marking, final_marking = im_f.algorithm.apply(model,parameters={"noiseThreshold" : 0.5})
    
    log = pd.read_csv(log, sep=',',encoding='latin1')
    log_csv= log[['RecordingId','ApplicationProcessNameStepName','TimeStamp']]

    cols = ['case:concept:name','concept:name','time:timestamp']

    log_csv.columns = cols
    log_csv['time:timestamp'] = pd.to_datetime(log_csv['time:timestamp'])
    log_csv['concept:name'] = log_csv['concept:name'].astype(str)

    log = log_converter.apply(log_csv, variant=log_converter.Variants.TO_EVENT_LOG)

    # Conformance checking using token-based replay
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking)

    df = pd.DataFrame(aligned_traces)
    # dff = df["trace_fitness"].mean()
    return df

In [ ]:
y = file_list_model
x = file_list_log
data_first0=[]
for i in file_list_model:
  row=[]

  for j in file_list_log:
     row.append(Conf_imf_align_NT(i,j))


  data_first0.append(row)
  

df_first0 = pd.DataFrame(data_first0, index=y, columns=x)
df_first0

In [ ]:
file_list_log

In [ ]:
trace=[]
fitness=[]
for k in range(10):
    trace.append(row[k].alignment)
    fitness.append(row[k].fitness)

In [ ]:
import numpy as np
print(x)
index = pd.MultiIndex.from_arrays([trace],names=x)

In [ ]:
# print((trace))
print(fitness)

[0    0.619048
1    0.473684
2    0.222222
3    0.222222
4    0.111111
5    0.210526
Name: fitness, dtype: float64, 0    0.285714
1    0.529412
2    0.733333
3    0.615385
Name: fitness, dtype: float64, 0    0.235294
1    0.823529
2    0.540000
3    0.181818
4    0.800000
5    0.181818
6    0.181818
Name: fitness, dtype: float64, 0    0.720000
1    0.434783
2    0.555556
3    0.633333
4    0.416667
Name: fitness, dtype: float64, 0    0.631579
1    0.923077
2    0.222222
3    0.666667
Name: fitness, dtype: float64, 0    0.235294
1    0.200000
2    0.777778
3    0.434783
4    0.210526
Name: fitness, dtype: float64, 0    0.105263
1    0.375000
2    0.517241
3    0.350000
4    0.000000
Name: fitness, dtype: float64, 0    0.545455
1    0.500000
2    0.529412
3    0.523810
Name: fitness, dtype: float64, 0    0.080000
1    0.555556
2    0.500000
3    0.476190
4    0.291667
Name: fitness, dtype: float64]


In [ ]:
pd.DataFrame(fitness)

,0,1,2,3,4,5,6
fitness,0.619048,0.473684,0.222222,0.222222,0.111111,0.210526,NaN
fitness,0.285714,0.529412,0.733333,0.615385,NaN,NaN,NaN
fitness,0.235294,0.823529,0.540000,0.181818,0.800000,0.181818,0.181818
fitness,0.720000,0.434783,0.555556,0.633333,0.416667,NaN,NaN
fitness,0.631579,0.923077,0.222222,0.666667,NaN,NaN,NaN
fitness,0.235294,0.200000,0.777778,0.434783,0.210526,NaN,NaN
fitness,0.105263,0.375000,0.517241,0.350000,0.000000,NaN,NaN
fitness,0.545455,0.500000,0.529412,0.523810,NaN,NaN,NaN
fitness,0.080000,0.555556,0.500000,0.476190,0.291667,NaN,NaN


In [ ]:
print(row[0])

[                                           alignment    cost  visited_states  \
0  [(>>, Press button in window), (>>, firefoxPop...   80017             259   
1  [(>>, Press button in window), (>>, firefoxPop...  100019             147   
2  [(Press button in window, Press button in wind...   70010              18   
3  [(Press button in window, Press button in wind...   70010              18   
4  [(ShellExperienceHostPress button in window, >...  160010              28   
5  [(Press button in window, Press button in wind...  150010              27   

   queued_states  traversed_arcs  lp_solved   fitness     bwc  
0            423             708        258  0.619048  210010  
1            302             390        137  0.473684  190010  
2             36              38          3  0.222222   90010  
3             36              38          2  0.222222   90010  
4             75              78          6  0.111111  180010  
5             73              75          1  0.210526 

In [ ]:
df_first0.to_csv('out.csv')